# BC205: Algorithms for Bioinformatics.
## IV. Motif Discovery

### Christoforos Nikolaou

### Up to now we have seen how:
* We define a sequence motif
* We can search for known short motifs with a determined degree
of ambiguity
* We can estimate the existence of a motif in a sequence
* We can define the strength of the motif in the sequence in an
Entropy-based score

### Recap of results in code:
a. Creating of a PWM from a given sequnce

In [46]:
# Code here
import regex as re
f=open("files/gata.fa", 'r')
seqs = []
for line in f:
    x=re.match(">", line)
    if x == None:
        seqs.append(line.rstrip())
#
nuc = ['A', 'C', 'G', 'T']
profile=[[0 for i in range(len(seqs[0]))] for j in range(len(nuc))]


for instance in seqs:
    for j in range(len(instance)):
        residue=instance[j]
        profile[nuc.index(residue)][j]+=1
        profile[nuc.index(residue)][j]=float(profile[nuc.index(residue)][j])

import numpy as np 
pwm = np.array(profile)
pwm = pwm/len(seqs)
print(pwm)
    


[[0.6  0.02 0.96 0.02 0.72 0.44]
 [0.04 0.02 0.02 0.   0.06 0.06]
 [0.02 0.9  0.02 0.04 0.04 0.4 ]
 [0.34 0.06 0.   0.94 0.18 0.1 ]]


b. Transformation of PWM into PSSM on the basis of a given sequence

We can transform this table by applying a normalization and log-transformation against nucleotide occurrences from a given sequence.

We shall first write a function to read the sequence in fasta format

In [94]:
def readfasta(fastafile):
    import regex as re
    f=open(fastafile, 'r')
    seq = ""
    total = 0
    for line in f:
        x=re.match(">", line)
        if x == None:
            length=len(line)
            total=total+length
            seq=seq+line[0:length-1]
    seq=seq.replace('N','')
    f.close()
    return(seq)

targetsequence=readfasta("files/ecoli.fa")


And then write another function to calculate nucleotide frequencies

In [95]:

def nuccomp(sequence):
    nucfreq = [0, 0, 0, 0]
    nuc = ['A', 'C', 'G', 'T']
    for i in range(len(nuc)):
        nucfreq[i]=sequence.count(nuc[i])
    nucfreq=np.array(nucfreq)/len(sequence)
    return(nucfreq)

nucfreqs=nuccomp(targetsequence)

Τhe next obvious step is to combine the PWM with the Array of the nucleotide composition of the target sequnce and log-transform the resulting table into a PSSM

In [96]:
# Creation of a PSSM
def pssm(pwm, nucfreqs):
    import numpy as np
    import math
    pseudocount=0.01
    pssm=[[0 for i in range(len(pwm[0]))] for j in range(len(nucfreqs))]
    for i in range(len(nucfreqs)):
        pssm[i]=(np.array(pwm[i])+pseudocount)/nucfreqs[i]
    for i in range(len(pssm)):
        for k in range(len(pssm[0])):
            pssm[i][k]=math.log(pssm[i][k])/math.log(2)
    return(pssm)

mypssm=pssm(pwm, nucfreqs)

In [ ]:
c. Searching a sequence with a PWM/PSSM or with Hamming Distance 

In [101]:
def pssmSearch(pssm, sequence, threshold):
    nuc = ['A', 'C', 'G', 'T']
    hits = []
    for i in range(len(sequence)-len(pssm[0])):
        instance=sequence[i:i+len(pssm[0])]
        score=0
        for l in range(len(instance)):
            score=score+pssm[nuc.index(instance[l])][l]
        if (score > threshold):
            value = str(i)+":"+instance+":"+str(score)
            hits.append(value) 
    return(hits)

pssmSearch(mypssm, targetsequence, 9)

,
 '948854:AGATAA:9.519952394511247',
 '950052:AGATAA:9.519952394511247',
 '954444:AGATAG:9.34069335937505',
 '957626:AGATAG:9.34069335937505',
 '958815:AGATAG:9.34069335937505',
 '964463:AGATAA:9.519952394511247',
 '974812:AGATAA:9.519952394511247',
 '980840:AGATAG:9.34069335937505',
 '984235:AGATAA:9.519952394511247',
 '985462:AGATAA:9.519952394511247',
 '988712:AGATAA:9.519952394511247',
 '991508:AGATAA:9.519952394511247',
 '991886:AGATAA:9.519952394511247',
 '992315:AGATAA:9.519952394511247',
 '993588:AGATAG:9.34069335937505',
 '994466:AGATAG:9.34069335937505',
 '998564:AGATAA:9.519952394511247',
 '1000258:AGATAA:9.519952394511247',
 '1000300:AGATAA:9.519952394511247',
 '1000671:AGATAA:9.519952394511247',
 '1000762:AGATAG:9.34069335937505',
 '1003149:AGATAA:9.519952394511247',
 '1007243:AGATAA:9.519952394511247',
 '1011117:AGATAG:9.34069335937505',
 '1012639:AGATAA:9.519952394511247',
 '1017303:AGATAA:9.519952394511247',
 '1020542:AGATAA:9.519952394511247',
 '1023776:AGATAA:9.51995

In [89]:
print((mypssm[3][6]))

IndexError: index 6 is out of bounds for axis 0 with size 6

d. Entropy calculations

### The next problem. Discover a new motif from a given set of sequences

#### Part 1. Formulating the problem
1. Given a set of sequences that each contains an instance of the motif, find the motif.



In [ ]:
# code here

1. Given a set of s sequences: Find a set of k-mers (for a given
length k, one from each sequence) that maximizes the score (or
minimizes the distance) of each (one) k-mer with its sequence
2. Collect k-mers
3. Create a motif from them


In [ ]:
# code

### Brute Force Approach

What is the complexity of the BFA?
1. Number of k-mers 4k
2. Number of k-mers in each sequence: (n − k + 1)
3. Number of calculations for each k-mer given s sequences of
length n: (n − k + 1)s
4. Total number of calculations 4k (n − k + 1)s

The complexity of the algorithm is at least O(ns ).

We need something faster!


In [ ]:
# code

* Assuming we have a way to calculate the distance of a k-mer k

```
from a given sequence seq  
    for k in kmers:  
        for seq in sequences:  
            if distance(k, seq)<min_distance:  
                min_distance<-distance(k,seq)  
                motif[seq]<-k
```

* Because each k-mer needs to pass only once through each
sequence, the median string has O(4k ) complexity because k is
(usually) much shorter than the length of the sequence.

* However, it is still quite slow and for k>10 its implementation
is still unapplicable.


In [ ]:
# code

### An alternative

* Assume a greedy approach to go through all sequences
updating a motif every time

* Starting from sequence i:

1. find the most common k-mer
2. create a profile from it (adding pseudocounts to all 0-values)
3. go to the next sequence
4. choose the k-mer that best fits the profile
5. store that k-mer in the collection and update profile
6. iterate steps 3->5.

* We’ ve just described a Greedy approach for discovering a
motif p of a given length k among t sequences.


In [ ]:
# code

### A greedy Approach

* Assuming a set of s sequences and a given consensus k-mer k:

* We will construct a PWM “on the go” as we move from one
sequence to the next.
1. For i=1 :
2. For each k in seq_i:  
    2.1 For i = 2 to i = s:  
    2.2 Find the best (smallest distance) kmer in seq_i  
    2.3 Build a profile  
    2.4 If the score(profile) is better than all previous update profile  
3. Repeat


### Analysis. Greedy Approach

1. Why Greedy: It takes kmers from the first sequence only to
scan in the following. Thus it doesn’t go through all
combinations of sequences and k-mers. As we’ve seen above
the trade-off is speed.
2. KEY: It assumes that all sequences contain the motif. If the
first sequence doesn’t contain the motif (in any variation) then
we are doomed in looking for something that is non-sensical.
3. A way to go around this is to sample a small percentage of
sequences randomly, which brings us to the next-to-last chapter
of the motif finding problem


### A randomized approach

* In the Greedy Approach we take the kmers from the first
sequence and scan over the rest. In this way an initial wrong
choice may lead you to disastrous results.
* In a Randomized Approach we start, instead with a
collection of s k-mers, one from each sequence, build a profile,
scan the sequences with that profile, update it and repeat until
the k-mer set is good enough match for the updated profile.
* Stop and think of the problems we get rid of with this
approach.


### Pseudocode

```
for seq in sequences:  
    profile[seq]<-random(k, seq)  
    while distance(profile, sequences)>threshold  
        for seq in sequences:  
        profile[seq]<-max(k, profile, seq)  
```

In [ ]:
# code